# **CoronaBERT Text Encoding**

In this colab, I will be getting the text encodings for the queries and documents, using CoronaBERT.  These will be saved for future reference.

## **Setup**

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 47.8 MB/s 
     |████████████████████████████████| 67 kB 3.9 MB/s 
     |████████████████████████████████| 3.3 MB 48.5 MB/s 
     |████████████████████████████████| 596 kB 70.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install --upgrade python-terrier

     |████████████████████████████████| 97 kB 3.1 MB/s 
     |████████████████████████████████| 1.1 MB 11.3 MB/s 
     |████████████████████████████████| 69 kB 10.8 MB/s 
     |████████████████████████████████| 255 kB 60.6 MB/s 
     |████████████████████████████████| 41 kB 433 kB/s 
     |████████████████████████████████| 1.8 MB 55.4 MB/s 
     |████████████████████████████████| 126 kB 58.9 MB/s 
     |████████████████████████████████| 6.4 MB 42.9 MB/s 
     |████████████████████████████████| 72 kB 2.1 MB/s 
     |████████████████████████████████| 294 kB 75.9 MB/s 
     |████████████████████████████████| 291 kB 75.7 MB/s 
     |████████████████████████████████| 59 kB 9.6 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=780b2ccbec4b892454eaceb37edb764d95c31bd474db81f7c9f51bce06178c30
  Stored in directory: /root/.cache/pip/wheels/a8/ec/0c/fd9fcb409ad158d1f62534f9b1924b406f

In [ ]:
import pyterrier as pt

if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
import pandas as pd
import pickle

In [ ]:
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = cord19.get_topics('title')

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [7.13MB/s]


In [ ]:
!wget https://github.com/DavidONeill75101/level-4-project/blob/master/Datasets/CORD-19_Datasets/round5_docs.csv?raw=true
round5_docs = pd.read_csv('/content/round5_docs.csv?raw=true').drop(columns=['Unnamed: 0'])

## **Prepare for Encoding**

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jakelever/coronabert")
model = AutoModelForSequenceClassification.from_pretrained("jakelever/coronabert",output_hidden_states=True)

model = model.to(device)

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/220k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

The following function will encode the query concatenated to the document with a '\n' character in between.

In [ ]:
def text_to_embed(text):
  # Tokenize it with appropriate padding and truncation
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=500)

  # Move the IDs of the tokens over to the GPU
  input_ids = inputs['input_ids'].to(device)

  # Run the model on the data
  outputs = model(input_ids=input_ids)

  # Extract the embeddings
  with torch.no_grad():
    # Get the final layer of the neural network, and average the embedding for all the tokens
    # Some researchers use the vector just for the first or final token of the sentence
    # instead of an average. I don't think there is a definitive best approach.
    # You could stick to the mean for now.
    embed = outputs.hidden_states[-1].squeeze().mean(axis=0)

    # Return the embedding to the CPU and convert to a numpy array
    embed = embed.cpu().numpy()

  return embed

## **Get Query Embeddings**

In [ ]:
query_embeddings = {}
for i in range(1, 51):
  query_embeddings[str(i)] = text_to_embed(topics.iloc[i-1]['query'])


# **Get Document Embeddings**

In [ ]:
text = round5_docs['title']+ '\n' + round5_docs['abstract']
text = text.apply(str)
round5_docs['text'] = text


In [ ]:
round5_docs['embedding'] = round5_docs['text'].apply(text_to_embed)

In [ ]:
doc_embeddings = {}

for docno, embedding in zip(round5_docs['docno'], round5_docs['embedding']):
  doc_embeddings[docno] = embedding